In [ ]:
import sys

In [ ]:
sys.path.insert(0,"../python/")
import analyzer_utils as au

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import pprint
import json

In [ ]:
import search_utils as su
from twitter_crawler import TwitterCrawler
from twython import Twython

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
from bokeh.io import output_notebook
output_notebook()

In [ ]:
time_frame = 60
max_request_per_time_frame = 10

In [ ]:
tcs = TwitterCrawler(time_frame=time_frame,max_requests=max_request_per_time_frame)
tcs.authenticate("../api_key.json")

In [ ]:
query = "#fina"

In [ ]:
search_params = {
    #"q": query,
    "geocode": "47.5410857,19.0594822,50km",
    "result_type": 'recent',
    "count" :100
}

tcs.set_search_arguments(search_args=search_params)


In [ ]:
coll_name = "dunaarena"
coll, db = au.get_coll(coll_name)
tcs.connect(coll_name)

In [ ]:
my_created_at="Mon Jul 11 23:59:59 +0000 2017"
max_id = 885787586589073410

def my_time_bound_filter(tweet):
    return su.time_bound_filter(tweet, created_at=my_created_at)

with open("../api_key.json", "r") as f:
    auth_info = json.load(f)
twitter_api = Twython(auth_info["api_key"], auth_info["api_secret"])

tweets = twitter_api.search(**search_params)

tweets

In [ ]:
tcs.search_by_query(wait_for=3, current_max_id=max_id, term_func=my_time_bound_filter)

In [ ]:
db.command("collstats", coll_name)["count"]

In [ ]:
for item in coll.find():
    pprint.pprint(item['entities']['hashtags'])
    break

In [ ]:
hashtags = [item["entities"]["hashtags"] for item in coll.find()]
hashtags

In [ ]:
set(map(lambda d: d["text"], hashtags[8]))

In [ ]:
hashtags = sum(hashtags, [])
hashtags

In [ ]:
hashtags = [d["text"] for d in hashtags]

In [ ]:
pd.Series(hashtags).value_counts()

In [ ]:
coordinates_bounded = [item["coordinates"] for item in coll.find()]
coordinates_bounded = list(filter(lambda c: c is not None, coordinates_bounded))
print(len(coordinates_bounded))
coordinates_bounded

In [ ]:
spam = set(["Pussy", "Lesbian", "Amateur", "BigTits", "Nipples", "Blowjob", "Milf", "Teen"])

In [ ]:
def get_intersection(hashtag):
    return spam.intersection( set(map(lambda d: d["text"], hashtag)) )

def check_hashtags(hashtag):
    if len(hashtag) == 0:
        return False
    elif len(get_intersection(hashtag)) == 0:
        return False
    else:
        return True

spam_tweets = [item for item in coll.find() if check_hashtags(item["entities"]["hashtags"])]
spam_tweets

In [ ]:
set([len(get_intersection(item["entities"]["hashtags"])) for item in spam_tweets])

In [ ]:
len(spam_tweets)

In [ ]:
[item["text"] for item in spam_tweets]

In [ ]:
pd.Series([item["user"]["id"] for item in spam_tweets]).value_counts()

In [ ]:
[item for item in spam_tweets if item["user"]["id"] == 2621820576][0]

In [ ]:
hashtags = [item["entities"]["hashtags"] for item in coll.find() if item not in spam_tweets]
hashtags = sum(hashtags, [])
hashtags = [d["text"] for d in hashtags]
pd.Series(hashtags).value_counts()